In [30]:
import numpy as np

num_bodies = 100
num_dimensions = 3
G = 6.67430e-11  # gravitational constant
dt = 0.01  # time step
num_steps = 100

def getAcc(positions, mass): 
    # positions r = [x,y,z] for all particles
    x = positions[:, 0:1]
    y = positions[:, 1:2]
    z = positions[:, 2:3]
    
    # matrix that stores all pairwise particle separations: r_j - r_i
    dx = x.T - x
    dy = y.T - y
    dz = z.T - z  
    
    # matrix that stores 1/r^3 for all particle pairwise particle separations 
    inv_r3 = (dx**2 + dy**2 + dz**2 + 0.01**2)
    inv_r3[inv_r3>0] = inv_r3[inv_r3>0]**(-1.5)
    ax = G * np.matmul((dx * inv_r3), mass)
    ay = G * np.matmul((dy * inv_r3), mass)
    az = G * np.matmul((dz * inv_r3), mass)
    
    # pack together the acceleration components
    a = np.hstack((ax, ay, az))
    return a

def simulate_n_body(v0, positions, mass) -> None:
    # Convert to Center-of-Mass frame
    vel = v0 - np.mean(mass * v0, 0) / np.mean(mass)
    # calculate initial gravitational accelerations
    acc = getAcc(positions, mass)
    
    for _ in range(num_steps):
        # (1/2) kick
        vel += acc * dt/2.0
        # drift
        positions += vel * dt
        # update accelerations
        acc = getAcc(positions, mass)
        # (1/2) kick
        vel += acc * dt/2.0
    
    return positions

In [31]:
import cupy as cp

num_bodies = 100
num_dimensions = 3
G = 6.67430e-11  # gravitational constant
dt = 0.01  # time step
num_steps = 100

def cp_getAcc(positions, mass): 
    # positions r = [x,y,z] for all particles
    x = positions[:, 0:1]
    y = positions[:, 1:2]
    z = positions[:, 2:3]
    
    # matrix that stores all pairwise particle separations: r_j - r_i
    dx = x.T - x
    dy = y.T - y
    dz = z.T - z  
    
    # matrix that stores 1/r^3 for all particle pairwise particle separations 
    inv_r3 = (dx**2 + dy**2 + dz**2 + 0.01**2)
    inv_r3[inv_r3>0] = inv_r3[inv_r3>0]**(-1.5)
    ax = G * cp.matmul((dx * inv_r3), mass)
    ay = G * cp.matmul((dy * inv_r3), mass)
    az = G * cp.matmul((dz * inv_r3), mass)
    
    # pack together the acceleration components
    a = cp.hstack((ax, ay, az))
    return a

def cp_simulate_n_body(v0, positions, mass) -> None:
    # Convert to Center-of-Mass frame
    vel = v0 - cp.mean(mass * v0, 0) / cp.mean(mass)
    # calculate initial gravitational accelerations
    acc = getAcc(positions, mass)
    
    for _ in range(num_steps):
        # (1/2) kick
        vel += acc * dt/2.0
        # drift
        positions += vel * dt
        # update accelerations
        acc = getAcc(positions, mass)
        # (1/2) kick
        vel += acc * dt/2.0
    
    return positions

In [32]:
# Example usage
num_bodies = 100
num_dimensions = 3
G = 6.67430e-11  # gravitational constant
dt = 0.01  # time step
num_steps = 100

positions = np.random.rand(num_bodies, num_dimensions)
velocities = np.random.rand(num_bodies, num_dimensions)
masses = 20.0 * np.ones((num_bodies, 1)) / num_bodies

# copy of the same data to device
pos_gpu = cp.asarray(positions)
vel_gpu = cp.asarray(velocities)
masses_gpu = cp.asarray(masses)

# Generate random initial positions, velocities
def cupy_run(positions, velocities, masses):
    # Run the simulation
    final_positions = cp_simulate_n_body(velocities, positions, masses)

    return final_positions

def numpy_run(positions, velocities, masses):
    # Run the simulation
    final_positions = simulate_n_body(velocities, positions, masses)

    return final_positions

In [33]:
# %%timeit
cp_pos = cupy_run(pos_gpu, vel_gpu, masses_gpu)

In [34]:
# %%timeit
np_pos = numpy_run(positions, velocities, masses)

In [35]:
cpu_cp_pos = cp_pos.get()

In [37]:
# np.allclose(np_pos, cpu_cp_pos)
np_pos[:10], cp_pos[:10]

(array([[ 0.71462362,  0.44611156,  0.07632734],
        [ 0.46017152,  0.67906261,  1.24123734],
        [ 0.50912575, -0.16336361,  1.12953872],
        [ 1.15205822,  1.11445925,  0.65428182],
        [ 0.51368936,  0.5666929 ,  0.83880671],
        [-0.31488647,  1.06231082,  0.7775603 ],
        [ 0.45796346,  0.85237976,  0.2778193 ],
        [ 1.15397985,  0.45546538,  0.7553989 ],
        [ 0.94280834,  0.22725628,  0.57380995],
        [ 0.50348746, -0.08487701,  0.34109575]]),
 array([[ 0.71462362,  0.44611156,  0.07632734],
        [ 0.46017152,  0.67906261,  1.24123734],
        [ 0.50912575, -0.16336361,  1.12953872],
        [ 1.15205822,  1.11445925,  0.65428182],
        [ 0.51368936,  0.5666929 ,  0.83880671],
        [-0.31488647,  1.06231082,  0.7775603 ],
        [ 0.45796346,  0.85237976,  0.2778193 ],
        [ 1.15397985,  0.45546538,  0.7553989 ],
        [ 0.94280834,  0.22725628,  0.57380995],
        [ 0.50348746, -0.08487701,  0.34109575]]))

## Agnostic code

In [26]:
def agnostic_run(positions, v0, mass):
    # copy data to device
    xp = cp.get_array_module(positions)
    print(f'using: {xp.__name__}')

    def xp_getAcc(positions, mass):
        # positions r = [x,y,z] for all particles
        x = positions[:, 0:1]
        y = positions[:, 1:2]
        z = positions[:, 2:3]

        # matrix that stores all pairwise particle separations: r_j - r_i
        dx = x.T - x
        dy = y.T - y
        dz = z.T - z  

        # matrix that stores 1/r^3 for all particle pairwise particle separations 
        inv_r3 = (dx**2 + dy**2 + dz**2 + 0.01**2)
        inv_r3[inv_r3>0] = inv_r3[inv_r3>0]**(-1.5)
        ax = G * xp.matmul((dx * inv_r3), mass)
        ay = G * xp.matmul((dy * inv_r3), mass)
        az = G * xp.matmul((dz * inv_r3), mass)

        # pack together the acceleration components
        a = xp.hstack((ax, ay, az))
        
        return a

    # Convert to Center-of-Mass frame
    vel = v0 - xp.mean(mass * v0, 0) / xp.mean(mass)
    
    # calculate initial gravitational accelerations
    acc = xp_getAcc(positions, mass)
    
    for _ in range(num_steps):
        # (1/2) kick
        vel += acc * dt/2.0
        # drift
        positions += vel * dt
        # update accelerations
        acc = xp_getAcc(positions, mass)
        # (1/2) kick
        vel += acc * dt/2.0

    return positions

In [27]:
ag_pos = agnostic_run(pos_gpu, vel_gpu, masses_gpu)

using: cupy


In [28]:
cpu_cp_pos = ag_pos.get()

In [29]:
np.allclose(np_pos, cpu_cp_pos)

False